In [1]:
from itertools import zip_longest
import sys

def unwrap(iterator):
    '''Get the sole element of iterator or raise a ValueError'''
    i = iter(iterator)
    try:
        out = next(i)
    except StopIteration:
        raise ValueError("i has no elements")
    try:
        next(i)
    except StopIteration:
        return out
    raise ValueError("i has more than one element")

In [3]:
class ResDiheds(object):
    def __init__(self, bonds, atoms, overrides, special_atoms=None):
        self.bonds = bonds
        self.atoms = atoms
        self.overrides = self._set_overrides(overrides)
        
        self.dihedrals = self.get_all_dihedrals(bonds)
        self.atomtypes = dict([s.split()[:2] for s in atoms.strip().splitlines()])
        for special_atom, special_atomtype in special_atoms:
            self.atomtypes[special_atom] = special_atomtype
        self.atomtypes['-C'] = 'C'
        self.atomtypes['+N'] = 'N'
    
    @staticmethod
    def _set_overrides(overrides):
        overrides_out = {}
        for line in overrides.strip().splitlines():
            line = line.strip()
            if line and not line.startswith(';'):
                splits = tuple(line.split())
                key = splits[:4]
                val = splits[4:]
                overrides_out.setdefault(key, []).append(val)
        return overrides_out
                
    def override(self, i,j,k,l):
        if (i,j,k,l) in self.overrides:
            key = (i,j,k,l)
        elif (l,k,j,i) in self.overrides:
            key = (l,k,j,i)
        elif ('X',k,j,'X') in self.overrides:
            key = ('X',k,j,'X')
        else:
            return None
        return self.paramname(key)
    
    def get_overrides(self):
        for dih in self.dihedrals:
            i,j,k,l = dih
            it, jt, kt, lt = (self.atomtypes[s] for s in dih)

            params = self.override(it,jt,kt,lt)
            if params:
                yield dih, params
        
    def print_overrides(self, prep=lambda s: s.rjust(6), file=sys.stdout):
        dihedrals = self.get_overrides()
        print("\n [ dihedrals ]", file=file)

        for dih in dihedrals:
            (i,j,k,l), params = dih
            for param in params:
                print('', prep(i), prep(j), prep(k), prep(l), '    ', param, sep='', file=file)
                
                
    @staticmethod
    def get_all_dihedrals(bonds):    
        bonds = [set(s.split()) for s in bonds.strip().splitlines()]
        dihedrals = []
        for bond in bonds:
            j, k = bond
            j_bonds = [unwrap(b - set([j])) for b in bonds if j in b]
            k_bonds = [unwrap(b - set([k])) for b in bonds if k in b]
            for i in j_bonds:
                for l in k_bonds:
                    if i != k and l != j:
                        dihedrals.append([i,j,k,l])
        return dihedrals
    
    def paramname(self, key):
        return self._paramname(self.overrides, key)
    
    @staticmethod
    def _paramname(overrides, key):
        params = overrides[key]
        i,j,k,l = key
        n = len(params)
        if n == 1:
            return [f'torsion_{i}_{j}_{k}_{l}']
        else:
            return [f'torsion_{i}_{j}_{k}_{l}_mult{m}' for m, p in enumerate(params, 1)]
        
class FFResDiheds(ResDiheds):
    overrides = ''
    special_atoms = []
    
    def __init__(self, bonds, atoms, special_atoms=None):
        bonds += '+N C'
        special_atoms += self.special_atoms
        super().__init__(bonds, atoms, overrides=self.overrides, special_atoms=special_atoms)
    
    @classmethod
    def print_override_defs(cls):
        for key, params in cls._set_overrides(cls.overrides).items():
            for name, param in zip(cls.paramname(key), params):
                print(f'#define {name.ljust(30)} ', *(s.ljust(8) for s in param[1:]))
            
    @classmethod
    def paramname(cls, key):
        overrides = cls._set_overrides(cls.overrides)
        return cls._paramname(overrides, key)
                
                
class A99SB_Disp(FFResDiheds):
    overrides = """ 
        ; Met, Gln, Glu, Arg, Lys overrides
         N    CT    CT    C9    9      0.00    0.1       1

         C    CT    CT    C9    9      0.00    0.3       1

        CT    CT    C9     S    9      0.00    0.0       1

        CT    C9     C    O2    9      0.00    0.0       1
        CT    C9     C    O2    9      0.00   -0.6       2

        CT    C9     C     O    9      0.00    0.0       1
        CT    C9     C     O    9      0.00   -0.6       2

        CT    CT    C9    CT    9      0.00    0.0       1

        ; Ser, Thr, Val overrides
        CT    C8    OH    HO    9      0.00    0.75      1
        CT    C8    OH    HO    9      0.00    0.06      2
        CT    C8    OH    HO    9      0.00    0.20      3

        CT    C8    O3    HO    9      0.00    0.086     1
        CT    C8    O3    HO    9      0.00   -0.163     2
        CT    C8    O3    HO    9      0.00    0.1       3

        H1    C8    O3    HO    9      0.00    0.0       1
        H1    C8    O3    HO    9      0.00    0.0       2
        H1    C8    O3    HO    9      0.00    0.1       3

         N    CT    C8    CT    9      0.00    0.06      1
         N    CT    C8    CT    9      0.00   -0.5       2
         N    CT    C8    CT    9      0.00    0.15      3
         N    CT    C8    CT    9      0.00   -0.065     4

         N    CT    C8    OH    9      0.00   -0.25      1

         C    CT    C8    OH    9      0.00   -0.90      1

         N    CT    C8    O3    9      0.00   -0.08      1

         C    CT    C8    O3    9      0.00   -1.00      1

         X    CT    C8     X    9      0.00    0.0       1
         X    CT    C8     X    9      0.00    0.0       2
         X    CT    C8     X    9      0.00    0.16      3

        C8    CT     N     C    9      0.00    2.00      1
        C8    CT     N     C    9      0.00    2.00      2
        C8    CT     N     C    9      0.00    0.40      3

        C8    CT     C     N    9      0.00    0.20      1
        C8    CT     C     N    9      0.00    0.20      2
        C8    CT     C     N    9      0.00    0.40      3

         ; LEU ovverides
        CT    CT    C4    CT    9      0.00    0.0       1
        CT    CT    C4    CT    9      0.00    0.0       2
        CT    CT    C4    CT    9      0.00    0.15      3

        HC    CT    CT    C4    9      0.00    0.0       1
        HC    CT    CT    C4    9      0.00    0.0       2
        HC    CT    CT    C4    9      0.00    0.16      3

        CT    C4    CT    CT    9      0.00    0.0       1
        CT    C4    CT    CT    9      0.00    0.0       2
        CT    C4    CT    CT    9      0.00    0.15      3

        HC    C4    CT    CT    9      0.00    0.0       1
        HC    C4    CT    CT    9      0.00    0.0       2
        HC    C4    CT    CT    9      0.00    0.16      3

        HC    C4    CT    HC    9      0.00    0.0       1
        HC    C4    CT    HC    9      0.00    0.0       2
        HC    C4    CT    HC    9      0.00    0.15      3

        HC    CT    C4    CT    9      0.00    0.0       1
        HC    CT    C4    CT    9      0.00    0.0       2
        HC    CT    C4    CT    9      0.00    0.16      3

         C    CT    CT    C4    9      0.00    0.5714    1
         C    CT    CT    C4    9      0.00   -0.3583    2
         C    CT    CT    C4    9      0.00    0.1348    3

         ; ILE overrides
        CT    CT    CT    C3    9      0.00    0.0       1
        CT    CT    CT    C3    9      0.00    0.0       2
        CT    CT    CT    C3    9      0.00    0.15      3

        HC    CT    CT    C3    9      0.00    0.0       1
        HC    CT    CT    C3    9      0.00    0.0       2
        HC    CT    CT    C3    9      0.00    0.16      3

        HC    C3    CT    CT    9      0.00    0.0       1
        HC    C3    CT    CT    9      0.00    0.0       2
        HC    C3    CT    CT    9      0.00    0.16      3

        HC    C3    CT    HC    9      0.00    0.0       1
        HC    C3    CT    HC    9      0.00    0.0       2
        HC    C3    CT    HC    9      0.00    0.15      3

         N    CT    CT    C3    9      0.00    0.1951    1
         N    CT    CT    C3    9      0.00   -0.8459    2

        N3    CT    CT    C3    9      0.00    0.1951    1
        N3    CT    CT    C3    9      0.00   -0.8459    2

         ; ASN overrides
        HC    CT    C5     O    9      0.00    0.8       1
        HC    CT    C5     O    9      0.00    0.0       2
        HC    CT    C5     O    9      0.00   -0.08      3

         H     N    C5     O    9      0.00    2.0       1
         H     N    C5     O    9      0.00   -2.5       2

         N    C5    CT    CT    9      0.00   -1.0463    1
         N    C5    CT    CT    9      0.00   -0.181     2
         N    C5    CT    CT    9      0.00   -0.0354    3
         N    C5    CT    CT    9      0.00    0.1003    4
         N    C5    CT    CT    9      0.00    0.1298    5
         N    C5    CT    CT    9      0.00   -0.1061    6

         N    CT    CT    C5    9      0.00   -0.63      1

         C    CT    CT    C5    9      0.00    0.1707    1
         C    CT    CT    C5    9      0.00   -0.5957    2
         C    CT    CT    C5    9      0.00    0.1185    3
         C    CT    CT    C5    9      0.00   -0.4172    4
         C    CT    CT    C5    9      0.00    0.1042    5
         C    CT    CT    C5    9      0.00   -0.1007    6

        CT    CT    C5     O    9      0.00   -1.0       1

         ; ASP overrides
         N    CT    CT    C6    9      0.00   -2.881     1
         N    CT    CT    C6    9      0.00   -1.211     2
         N    CT    CT    C6    9      0.00    0.062     3
         N    CT    CT    C6    9      0.00    0.245     4
         N    CT    CT    C6    9      0.00    0.131     5
         N    CT    CT    C6    9      0.00   -0.142     6

        N3    CT    CT    C6    9      0.00   -2.6352    1
        N3    CT    CT    C6    9      0.00   -1.1903    2
        N3    CT    CT    C6    9      0.00   -0.068     3
        N3    CT    CT    C6    9      0.00    0.4227    4
        N3    CT    CT    C6    9      0.00    0.1042    5
        N3    CT    CT    C6    9      0.00   -0.2127    6

        CT    CT    C6    O2    9      0.00    0.0       1
        CT    CT    C6    O2    9      0.00   -0.873     2
        CT    CT    C6    O2    9      0.00    0.0       3
        CT    CT    C6    O2    9      0.00   -0.186     4
        CT    CT    C6    O2    9      0.00    0.0       5
        CT    CT    C6    O2    9      0.00   -0.354     6

         ; GLY overrides
         C     N    C1     C    9      0.00    0.084     1
         C     N    C1     C    9      0.00    0.381     2
         C     N    C1     C    9      0.00   -0.020     3
         C     N    C1     C    9      0.00    0.077     4

         N    C1     C     N    9      0.00   -0.644     1
         N    C1     C     N    9      0.00   -1.598     2
         N    C1     C     N    9      0.00   -0.399     3
         N    C1     C     N    9      0.00    0.075     4

        H1    C1     C     O    9      0.00    0.80      1
        H1    C1     C     O    9      0.00    0.0       2
        H1    C1     C     O    9      0.00   -0.08      3

         X    C1    N3     X    9      0.00    0.0       1
         X    C1    N3     X    9      0.00    0.156     2

         X    C1     N     X    9      0.00    0.0       1

        X C1 C X 9 0.00 0.0 1
    """
    special_atoms=[
        ('H', 'H'),
        ('O', 'O')
    ]
    

In [4]:
A99SB_Disp.print_override_defs()

#define torsion_N_CT_CT_C9              0.00     0.1      1       
#define torsion_C_CT_CT_C9              0.00     0.3      1       
#define torsion_CT_CT_C9_S              0.00     0.0      1       
#define torsion_CT_C9_C_O2_mult1        0.00     0.0      1       
#define torsion_CT_C9_C_O2_mult2        0.00     -0.6     2       
#define torsion_CT_C9_C_O_mult1         0.00     0.0      1       
#define torsion_CT_C9_C_O_mult2         0.00     -0.6     2       
#define torsion_CT_CT_C9_CT             0.00     0.0      1       
#define torsion_CT_C8_OH_HO_mult1       0.00     0.75     1       
#define torsion_CT_C8_OH_HO_mult2       0.00     0.06     2       
#define torsion_CT_C8_OH_HO_mult3       0.00     0.20     3       
#define torsion_CT_C8_O3_HO_mult1       0.00     0.086    1       
#define torsion_CT_C8_O3_HO_mult2       0.00     -0.163   2       
#define torsion_CT_C8_O3_HO_mult3       0.00     0.1      3       
#define torsion_H1_C8_O3_HO_mult1       0.00     0.0      1   

In [81]:
met = A99SB_Disp(
    bonds="""
         N     H
         N    CA
        CA    HA
        CA    CB
        CA     C
        CB   HB1
        CB   HB2
        CB    CG
        CG   HG1
        CG   HG2
        CG    SD
        SD    CE
        CE   HE1
        CE   HE2
        CE   HE3
         C     O
        -C     N
    """, 
    atoms="""
         N    N           -0.41570     1
         H    HN           0.27190     2
        CA    CT          -0.02370     3
        HA    H1           0.08800     4
        CB    CT           0.03420     5
       HB1    HC           0.02410     6
       HB2    HC           0.02410     7
        CG    CT           0.00180     8
       HG1    H1           0.04400     9
       HG2    H1           0.04400    10
        SD    S           -0.27370    11
        CE    CT          -0.05360    12
       HE1    H1           0.06840    13
       HE2    H1           0.06840    14
       HE3    H1           0.06840    15
         C    C            0.59730    16
         O    OC          -0.56790    17
    """, 
    special_atoms=[
        ('CG', 'C9')
    ]
)

met.print_overrides()


 [ dihedrals ]
     N    CA    CB    CG    torsion_N_CT_CT_C9
     C    CA    CB    CG    torsion_C_CT_CT_C9
    CA    CB    CG    SD    torsion_CT_CT_C9_S


In [82]:
GLN = A99SB_Disp(
    bonds="""
         N     H
         N    CA
        CA    HA
        CA    CB
        CA     C
        CB   HB1
        CB   HB2
        CB    CG
        CG   HG1
        CG   HG2
        CG    CD
        CD   OE1
        CD   NE2
       NE2  HE21
       NE2  HE22
         C     O
        -C     N
    """, 
    atoms="""
         N    N           -0.41570     1
         H    HN           0.27190     2
        CA    CT          -0.00310     3
        HA    H1           0.08500     4
        CB    CT          -0.00360     5
       HB1    HC           0.01710     6
       HB2    HC           0.01710     7
        CG    CT          -0.06450     8
       HG1    HC           0.03520     9
       HG2    HC           0.03520    10
        CD    C            0.69510    11
       OE1    O           -0.60860    12
       NE2    N           -0.94070    13
      HE21    H            0.42510    14
      HE22    H            0.42510    15
         C    C            0.59730    16
         O    OC          -0.56790    17
    """, 
    special_atoms=[
        ('CG', 'C9')
    ]
)

GLN.print_overrides()


 [ dihedrals ]
     N    CA    CB    CG    torsion_N_CT_CT_C9
     C    CA    CB    CG    torsion_C_CT_CT_C9
   OE1    CD    CG    CB    torsion_CT_C9_C_O_mult1
   OE1    CD    CG    CB    torsion_CT_C9_C_O_mult2


In [83]:
GLU = A99SB_Disp(
    bonds="""
         N     H
         N    CA
        CA    HA
        CA    CB
        CA     C
        CB   HB1
        CB   HB2
        CB    CG
        CG   HG1
        CG   HG2
        CG    CD
        CD   OE1
        CD   OE2
         C     O
        -C     N
    """, 
    atoms="""
         N    N        -0.415700        1 ; a99sb-disp
         H    HN        0.271900        2 ; a99sb-disp
        CA    CT       -0.227583        3 ; a99sb-disp
        HA    H1        0.154545        4 ; a99sb-disp
        CB    CT        0.116010        5 ; a99sb-disp
       HB1    HC       -0.006217        6 ; a99sb-disp
       HB2    HC       -0.006217        7 ; a99sb-disp
        CG    CT       -0.081794        8 ; a99sb-disp
       HG1    HC       -0.006154        9 ; a99sb-disp
       HG2    HC       -0.006154       10 ; a99sb-disp
        CD    C         0.728614       11 ; a99sb-disp
       OE1    O2       -0.775325       12 ; a99sb-disp
       OE2    O2       -0.775325       13 ; a99sb-disp
         C    C         0.597300       14 ; a99sb-disp
         O    OC       -0.567900       15 ; a99sb-disp
    """, 
    special_atoms=[
        ('CG', 'C9')
    ]
)

GLU.print_overrides()


 [ dihedrals ]
     N    CA    CB    CG    torsion_N_CT_CT_C9
     C    CA    CB    CG    torsion_C_CT_CT_C9
   OE1    CD    CG    CB    torsion_CT_C9_C_O2_mult1
   OE1    CD    CG    CB    torsion_CT_C9_C_O2_mult2
   OE2    CD    CG    CB    torsion_CT_C9_C_O2_mult1
   OE2    CD    CG    CB    torsion_CT_C9_C_O2_mult2


In [84]:
ARG = A99SB_Disp(
    bonds="""
         N     H
         N    CA
        CA    HA
        CA    CB
        CA     C
        CB   HB1
        CB   HB2
        CB    CG
        CG   HG1
        CG   HG2
        CG    CD
        CD   HD1
        CD   HD2
        CD    NE
        NE    HE
        NE    CZ
        CZ   NH1
        CZ   NH2
       NH1  HH11
       NH1  HH12
       NH2  HH21
       NH2  HH22
         C     O
        -C     N
    """, 
    atoms="""
         N    N       -0.415700       1    ; a99sb-disp
         H    HN       0.271900       2    ; a99sb-disp
        CA    CT       0.053892       3    ; a99sb-disp
        HA    H1       0.086610       4    ; a99sb-disp
        CB    CT       0.052535       5    ; a99sb-disp
       HB1    HC       0.023551       6    ; a99sb-disp
       HB2    HC       0.023551       7    ; a99sb-disp
        CG    CT      -0.000834       8    ; a99sb-disp
       HG1    HC       0.013058       9    ; a99sb-disp
       HG2    HC       0.013058      10    ; a99sb-disp
        CD    CT      -0.251845      11    ; a99sb-disp
       HD1    H1       0.217628      12    ; a99sb-disp
       HD2    H1       0.217628      13    ; a99sb-disp
        NE    N2      -0.684202      14    ; a99sb-disp
        HE    H        0.394341      15    ; a99sb-disp
        CZ    CA       1.034975      16    ; a99sb-disp
       NH1    N2      -0.812007      17    ; a99sb-disp
      HH11    H        0.386117      18    ; a99sb-disp
      HH12    H        0.386117      19    ; a99sb-disp
       NH2    N2      -0.812007      20    ; a99sb-disp
      HH21    H        0.386117      21    ; a99sb-disp
      HH22    H        0.386117      22    ; a99sb-disp
         C    C        0.597300      23    ; a99sb-disp
         O    OC      -0.567900      24    ; a99sb-disp
    """, 
    special_atoms=[
        ('CG', 'C9')
    ]
)

ARG.print_overrides()


 [ dihedrals ]
     N    CA    CB    CG    torsion_N_CT_CT_C9
     C    CA    CB    CG    torsion_C_CT_CT_C9
    CA    CB    CG    CD    torsion_CT_CT_C9_CT


In [85]:
lys = A99SB_Disp(
    bonds="""
         N     H
         N    CA
        CA    HA
        CA    CB
        CA     C
        CB   HB1
        CB   HB2
        CB    CG
        CG   HG1
        CG   HG2
        CG    CD
        CD   HD1
        CD   HD2
        CD    CE
        CE   HE1
        CE   HE2
        CE    NZ
        NZ   HZ1
        NZ   HZ2
        NZ   HZ3
         C     O
        -C     N
    """, 
    atoms="""
         N    N       -0.415700       1 ; a99sb-disp
         H    HN       0.271900       2 ; a99sb-disp
        CA    CT       0.105852       3 ; a99sb-disp
        HA    H1       0.078803       4 ; a99sb-disp
        CB    CT      -0.148415       5 ; a99sb-disp
       HB1    HC       0.063381       6 ; a99sb-disp
       HB2    HC       0.063381       7 ; a99sb-disp
        CG    CT       0.017428       8 ; a99sb-disp
       HG1    HC       0.022278       9 ; a99sb-disp
       HG2    HC       0.022278      10 ; a99sb-disp
        CD    CT      -0.099164      11 ; a99sb-disp
       HD1    HC       0.042554      12 ; a99sb-disp
       HD2    HC       0.042554      13 ; a99sb-disp
        CE    CT       0.235011      14 ; a99sb-disp
       HE1    HP       0.051928      15 ; a99sb-disp
       HE2    HP       0.051928      16 ; a99sb-disp
        NZ    N3      -0.594133      17 ; a99sb-disp
       HZ1    H        0.386245      18 ; a99sb-disp
       HZ2    H        0.386245      19 ; a99sb-disp
       HZ3    H        0.386245      20 ; a99sb-disp
         C    C        0.597300      21 ; a99sb-disp
         O    OC      -0.567900      22 ; a99sb-disp
    """, 
    special_atoms=[
        ('CG', 'C9')
    ]
)

lys.print_overrides()


 [ dihedrals ]
     N    CA    CB    CG    torsion_N_CT_CT_C9
     C    CA    CB    CG    torsion_C_CT_CT_C9
    CA    CB    CG    CD    torsion_CT_CT_C9_CT
    CE    CD    CG    CB    torsion_CT_CT_C9_CT


In [86]:
ser = A99SB_Disp(
    bonds="""
         N     H
         N    CA
        CA    HA
        CA    CB
        CA     C
        CB   HB1
        CB   HB2
        CB    OG
        OG    HG
         C     O
        -C     N
    """, 
    atoms="""
         N    N           -0.41570     1
         H    HN           0.27190     2
        CA    CT          -0.02490     3
        HA    H1           0.08430     4
        CB    CT           0.21170     5
       HB1    H1           0.03520     6
       HB2    H1           0.03520     7
        OG    OH          -0.65460     8
        HG    HO           0.42750     9
         C    C            0.59730    10
         O    OC          -0.56790    11
    """, 
    special_atoms=[
        ('CB', 'C8')
    ]
)

ser.print_overrides()


 [ dihedrals ]
    CB    CA     N    -C    torsion_C8_CT_N_C_mult1
    CB    CA     N    -C    torsion_C8_CT_N_C_mult2
    CB    CA     N    -C    torsion_C8_CT_N_C_mult3
     N    CA    CB    OG    torsion_N_CT_C8_OH
     C    CA    CB    OG    torsion_C_CT_C8_OH
    CB    CA     C    +N    torsion_C8_CT_C_N_mult1
    CB    CA     C    +N    torsion_C8_CT_C_N_mult2
    CB    CA     C    +N    torsion_C8_CT_C_N_mult3
    HG    OG    CB    CA    torsion_CT_C8_OH_HO_mult1
    HG    OG    CB    CA    torsion_CT_C8_OH_HO_mult2
    HG    OG    CB    CA    torsion_CT_C8_OH_HO_mult3


In [87]:
thr = A99SB_Disp(
    bonds="""
         N     H
         N    CA
        CA    HA
        CA    CB
        CA     C
        CB    HB
        CB   CG2
        CB   OG1
       CG2  HG21
       CG2  HG22
       CG2  HG23
       OG1   HG1
         C     O
        -C     N
    """, 
    atoms="""
         N    N           -0.41570     1
         H    HN           0.27190     2
        CA    CT          -0.03890     3
        HA    H1           0.10070     4
        CB    CT           0.36540     5
        HB    H1           0.00430     6
       CG2    CT          -0.24380     7
      HG21    HC           0.06420     8
      HG22    HC           0.06420     9
      HG23    HC           0.06420    10
       OG1    OH          -0.67610    11
       HG1    HO           0.41020    12
         C    C            0.59730    13
         O    OC          -0.56790    14
    """, 
    special_atoms=[
        ('CB', 'C8'),
        ('OG1', 'O3')
    ]
)

thr.print_overrides()


 [ dihedrals ]
    CB    CA     N    -C    torsion_C8_CT_N_C_mult1
    CB    CA     N    -C    torsion_C8_CT_N_C_mult2
    CB    CA     N    -C    torsion_C8_CT_N_C_mult3
     N    CA    CB   CG2    torsion_N_CT_C8_CT_mult1
     N    CA    CB   CG2    torsion_N_CT_C8_CT_mult2
     N    CA    CB   CG2    torsion_N_CT_C8_CT_mult3
     N    CA    CB   CG2    torsion_N_CT_C8_CT_mult4
     N    CA    CB   OG1    torsion_N_CT_C8_O3
     C    CA    CB   OG1    torsion_C_CT_C8_O3
    CB    CA     C    +N    torsion_C8_CT_C_N_mult1
    CB    CA     C    +N    torsion_C8_CT_C_N_mult2
    CB    CA     C    +N    torsion_C8_CT_C_N_mult3
    CA    CB   OG1   HG1    torsion_CT_C8_O3_HO_mult1
    CA    CB   OG1   HG1    torsion_CT_C8_O3_HO_mult2
    CA    CB   OG1   HG1    torsion_CT_C8_O3_HO_mult3
    HB    CB   OG1   HG1    torsion_H1_C8_O3_HO_mult1
    HB    CB   OG1   HG1    torsion_H1_C8_O3_HO_mult2
    HB    CB   OG1   HG1    torsion_H1_C8_O3_HO_mult3
   CG2    CB   OG1   HG1    torsion_CT_C8_

In [88]:
val = A99SB_Disp(
    bonds="""
         N     H
         N    CA
        CA    HA
        CA    CB
        CA     C
        CB    HB
        CB   CG1
        CB   CG2
       CG1  HG11
       CG1  HG12
       CG1  HG13
       CG2  HG21
       CG2  HG22
       CG2  HG23
         C     O
        -C     N
    """, 
    atoms="""
         N    N           -0.41570     1
         H    HN           0.27190     2
        CA    CT          -0.08750     3
        HA    H1           0.09690     4
        CB    CT           0.29850     5
        HB    HC          -0.02970     6
       CG1    CT          -0.31920     7
      HG11    HC           0.07910     8
      HG12    HC           0.07910     9
      HG13    HC           0.07910    10
       CG2    CT          -0.31920    11
      HG21    HC           0.07910    12
      HG22    HC           0.07910    13
      HG23    HC           0.07910    14
         C    C            0.59730    15
         O    OC          -0.56790    16
    """, 
    special_atoms=[
        ('CB', 'C8')
    ]
)

val.print_overrides()


 [ dihedrals ]
    CB    CA     N    -C    torsion_C8_CT_N_C_mult1
    CB    CA     N    -C    torsion_C8_CT_N_C_mult2
    CB    CA     N    -C    torsion_C8_CT_N_C_mult3
     N    CA    CB   CG1    torsion_N_CT_C8_CT_mult1
     N    CA    CB   CG1    torsion_N_CT_C8_CT_mult2
     N    CA    CB   CG1    torsion_N_CT_C8_CT_mult3
     N    CA    CB   CG1    torsion_N_CT_C8_CT_mult4
     N    CA    CB   CG2    torsion_N_CT_C8_CT_mult1
     N    CA    CB   CG2    torsion_N_CT_C8_CT_mult2
     N    CA    CB   CG2    torsion_N_CT_C8_CT_mult3
     N    CA    CB   CG2    torsion_N_CT_C8_CT_mult4
    CB    CA     C    +N    torsion_C8_CT_C_N_mult1
    CB    CA     C    +N    torsion_C8_CT_C_N_mult2
    CB    CA     C    +N    torsion_C8_CT_C_N_mult3


In [89]:
leu = A99SB_Disp(
    bonds="""
         N     H
         N    CA
        CA    HA
        CA    CB
        CA     C
        CB   HB1
        CB   HB2
        CB    CG
        CG    HG
        CG   CD1
        CG   CD2
       CD1  HD11
       CD1  HD12
       CD1  HD13
       CD2  HD21
       CD2  HD22
       CD2  HD23
         C     O
        -C     N
    """, 
    atoms="""
         N    N           -0.41570     1
         H    HN           0.27190     2
        CA    CT          -0.05180     3
        HA    H1           0.09220     4
        CB    CT          -0.11020     5
       HB1    HC           0.04570     6
       HB2    HC           0.04570     7
        CG    CT           0.35310     8
        HG    HC          -0.03610     9
       CD1    CT          -0.41210    10
      HD11    HC           0.10000    11
      HD12    HC           0.10000    12
      HD13    HC           0.10000    13
       CD2    CT          -0.41210    14
      HD21    HC           0.10000    15
      HD22    HC           0.10000    16
      HD23    HC           0.10000    17
         C    C            0.59730    18
         O    OC          -0.56790    19
    """, 
    special_atoms=[
        ('CG', 'C4')
    ]
)

leu.print_overrides()


 [ dihedrals ]
     C    CA    CB    CG    torsion_C_CT_CT_C4_mult1
     C    CA    CB    CG    torsion_C_CT_CT_C4_mult2
     C    CA    CB    CG    torsion_C_CT_CT_C4_mult3
    CA    CB    CG    HG    torsion_HC_C4_CT_CT_mult1
    CA    CB    CG    HG    torsion_HC_C4_CT_CT_mult2
    CA    CB    CG    HG    torsion_HC_C4_CT_CT_mult3
    CA    CB    CG   CD1    torsion_CT_CT_C4_CT_mult1
    CA    CB    CG   CD1    torsion_CT_CT_C4_CT_mult2
    CA    CB    CG   CD1    torsion_CT_CT_C4_CT_mult3
    CA    CB    CG   CD2    torsion_CT_CT_C4_CT_mult1
    CA    CB    CG   CD2    torsion_CT_CT_C4_CT_mult2
    CA    CB    CG   CD2    torsion_CT_CT_C4_CT_mult3
   HB1    CB    CG    HG    torsion_HC_C4_CT_HC_mult1
   HB1    CB    CG    HG    torsion_HC_C4_CT_HC_mult2
   HB1    CB    CG    HG    torsion_HC_C4_CT_HC_mult3
   HB1    CB    CG   CD1    torsion_HC_CT_C4_CT_mult1
   HB1    CB    CG   CD1    torsion_HC_CT_C4_CT_mult2
   HB1    CB    CG   CD1    torsion_HC_CT_C4_CT_mult3
   HB1    CB   

In [90]:
ile = A99SB_Disp(
    bonds="""
         N     H
         N    CA
        CA    HA
        CA    CB
        CA     C
        CB    HB
        CB   CG2
        CB   CG1
       CG2  HG21
       CG2  HG22
       CG2  HG23
       CG1  HG11
       CG1  HG12
       CG1    CD
        CD   HD1
        CD   HD2
        CD   HD3
         C     O
        -C     N
    """, 
    atoms="""
         N    N           -0.41570     1
         H    HN           0.27190     2
        CA    CT          -0.05970     3
        HA    H1           0.08690     4
        CB    CT           0.13030     5
        HB    HC           0.01870     6
       CG2    CT          -0.32040     7
      HG21    HC           0.08820     8
      HG22    HC           0.08820     9
      HG23    HC           0.08820    10
       CG1    CT          -0.04300    11
      HG11    HC           0.02360    12
      HG12    HC           0.02360    13
        CD    CT          -0.06600    14
       HD1    HC           0.01860    15
       HD2    HC           0.01860    16
       HD3    HC           0.01860    17
         C    C            0.59730    18
         O    OC          -0.56790    19
    """, 
    special_atoms=[
        ('CD', 'C3')
    ]
)

ile.print_overrides()


 [ dihedrals ]
    CD   CG1    CB    CA    torsion_CT_CT_CT_C3_mult1
    CD   CG1    CB    CA    torsion_CT_CT_CT_C3_mult2
    CD   CG1    CB    CA    torsion_CT_CT_CT_C3_mult3
    CD   CG1    CB    HB    torsion_HC_CT_CT_C3_mult1
    CD   CG1    CB    HB    torsion_HC_CT_CT_C3_mult2
    CD   CG1    CB    HB    torsion_HC_CT_CT_C3_mult3
    CD   CG1    CB   CG2    torsion_CT_CT_CT_C3_mult1
    CD   CG1    CB   CG2    torsion_CT_CT_CT_C3_mult2
    CD   CG1    CB   CG2    torsion_CT_CT_CT_C3_mult3
   HD1    CD   CG1    CB    torsion_HC_C3_CT_CT_mult1
   HD1    CD   CG1    CB    torsion_HC_C3_CT_CT_mult2
   HD1    CD   CG1    CB    torsion_HC_C3_CT_CT_mult3
   HD1    CD   CG1  HG11    torsion_HC_C3_CT_HC_mult1
   HD1    CD   CG1  HG11    torsion_HC_C3_CT_HC_mult2
   HD1    CD   CG1  HG11    torsion_HC_C3_CT_HC_mult3
   HD1    CD   CG1  HG12    torsion_HC_C3_CT_HC_mult1
   HD1    CD   CG1  HG12    torsion_HC_C3_CT_HC_mult2
   HD1    CD   CG1  HG12    torsion_HC_C3_CT_HC_mult3
   HD2    CD

In [91]:
asn = A99SB_Disp(
    bonds="""
         N     H
         N    CA
        CA    HA
        CA    CB
        CA     C
        CB   HB1
        CB   HB2
        CB    CG
        CG   OD1
        CG   ND2
       ND2  HD21
       ND2  HD22
         C     O
        -C     N
    """, 
    atoms="""
         N    N           -0.41570     1
         H    HN           0.27190     2
        CA    CT           0.01430     3
        HA    H1           0.10480     4
        CB    CT          -0.20410     5
       HB1    HC           0.07970     6
       HB2    HC           0.07970     7
        CG    C            0.71300     8
       OD1    O           -0.59310     9
       ND2    N           -0.91910    10
      HD21    H            0.41960    11
      HD22    H            0.41960    12
         C    C            0.59730    13
         O    OC          -0.56790    14
    """, 
    special_atoms=[
        ('CG', 'C5')
    ]
)

asn.print_overrides()


 [ dihedrals ]
     N    CA    CB    CG    torsion_N_CT_CT_C5
     C    CA    CB    CG    torsion_C_CT_CT_C5_mult1
     C    CA    CB    CG    torsion_C_CT_CT_C5_mult2
     C    CA    CB    CG    torsion_C_CT_CT_C5_mult3
     C    CA    CB    CG    torsion_C_CT_CT_C5_mult4
     C    CA    CB    CG    torsion_C_CT_CT_C5_mult5
     C    CA    CB    CG    torsion_C_CT_CT_C5_mult6
    CA    CB    CG   OD1    torsion_CT_CT_C5_O
    CA    CB    CG   ND2    torsion_N_C5_CT_CT_mult1
    CA    CB    CG   ND2    torsion_N_C5_CT_CT_mult2
    CA    CB    CG   ND2    torsion_N_C5_CT_CT_mult3
    CA    CB    CG   ND2    torsion_N_C5_CT_CT_mult4
    CA    CB    CG   ND2    torsion_N_C5_CT_CT_mult5
    CA    CB    CG   ND2    torsion_N_C5_CT_CT_mult6
   HB1    CB    CG   OD1    torsion_HC_CT_C5_O_mult1
   HB1    CB    CG   OD1    torsion_HC_CT_C5_O_mult2
   HB1    CB    CG   OD1    torsion_HC_CT_C5_O_mult3
   HB2    CB    CG   OD1    torsion_HC_CT_C5_O_mult1
   HB2    CB    CG   OD1    torsion_HC_CT_

In [92]:
asp = A99SB_Disp(
    bonds="""
         N     H
         N    CA
        CA    HA
        CA    CB
        CA     C
        CB   HB1
        CB   HB2
        CB    CG
        CG   OD1
        CG   OD2
         C     O
        -C     N
    """, 
    atoms="""
         N    N          -0.415700      1 ; a99sb-disp
         H    HN          0.271900      2 ; a99sb-disp
        CA    CT         -0.224564      3 ; a99sb-disp
        HA    H1          0.146193      4 ; a99sb-disp
        CB    CT          0.003572      5 ; a99sb-disp
       HB1    HC         -0.015579      6 ; a99sb-disp
       HB2    HC         -0.015579      7 ; a99sb-disp
        CG    C           0.762867      8 ; a99sb-disp
       OD1    O2         -0.771255      9 ; a99sb-disp
       OD2    O2         -0.771255     10 ; a99sb-disp
         C    C           0.597300     11 ; a99sb-disp
         O    OC         -0.567900     12 ; a99sb-disp
    """, 
    special_atoms=[
        ('CG', 'C6')
    ]
)

asp.print_overrides()


 [ dihedrals ]
     N    CA    CB    CG    torsion_N_CT_CT_C6_mult1
     N    CA    CB    CG    torsion_N_CT_CT_C6_mult2
     N    CA    CB    CG    torsion_N_CT_CT_C6_mult3
     N    CA    CB    CG    torsion_N_CT_CT_C6_mult4
     N    CA    CB    CG    torsion_N_CT_CT_C6_mult5
     N    CA    CB    CG    torsion_N_CT_CT_C6_mult6
    CA    CB    CG   OD1    torsion_CT_CT_C6_O2_mult1
    CA    CB    CG   OD1    torsion_CT_CT_C6_O2_mult2
    CA    CB    CG   OD1    torsion_CT_CT_C6_O2_mult3
    CA    CB    CG   OD1    torsion_CT_CT_C6_O2_mult4
    CA    CB    CG   OD1    torsion_CT_CT_C6_O2_mult5
    CA    CB    CG   OD1    torsion_CT_CT_C6_O2_mult6
    CA    CB    CG   OD2    torsion_CT_CT_C6_O2_mult1
    CA    CB    CG   OD2    torsion_CT_CT_C6_O2_mult2
    CA    CB    CG   OD2    torsion_CT_CT_C6_O2_mult3
    CA    CB    CG   OD2    torsion_CT_CT_C6_O2_mult4
    CA    CB    CG   OD2    torsion_CT_CT_C6_O2_mult5
    CA    CB    CG   OD2    torsion_CT_CT_C6_O2_mult6


In [93]:
gly = A99SB_Disp(
    bonds="""
         N     H
         N    CA
        CA   HA1
        CA   HA2
        CA     C
         C     O
        -C     N
    """, 
    atoms="""
         N    N           -0.41570     1
         H    HN           0.27190     2
        CA    CT          -0.02520     3
       HA1    H1           0.06980     4
       HA2    H1           0.06980     5
         C    C            0.59730     6
         O    OC          -0.56790     7
    """, 
    special_atoms=[
        ('CA', 'C1')
    ]
)

gly.print_overrides()


 [ dihedrals ]
     C    CA     N    -C    torsion_C_N_C1_C_mult1
     C    CA     N    -C    torsion_C_N_C1_C_mult2
     C    CA     N    -C    torsion_C_N_C1_C_mult3
     C    CA     N    -C    torsion_C_N_C1_C_mult4
     N    CA     C    +N    torsion_N_C1_C_N_mult1
     N    CA     C    +N    torsion_N_C1_C_N_mult2
     N    CA     C    +N    torsion_N_C1_C_N_mult3
     N    CA     C    +N    torsion_N_C1_C_N_mult4
   HA1    CA     C     O    torsion_H1_C1_C_O_mult1
   HA1    CA     C     O    torsion_H1_C1_C_O_mult2
   HA1    CA     C     O    torsion_H1_C1_C_O_mult3
   HA2    CA     C     O    torsion_H1_C1_C_O_mult1
   HA2    CA     C     O    torsion_H1_C1_C_O_mult2
   HA2    CA     C     O    torsion_H1_C1_C_O_mult3
